In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
!pip install comet_ml
from comet_ml import Experiment
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import joblib
import time
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [3]:
# Importo los datos
features = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50_PCA95.joblib"))
label = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib"))

%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"
# Importo particiones
indices_train = np.array(joblib.load('indices_train_W200_I50_sujeto.pkl'))
indices_val = np.array(joblib.load('indices_val_W200_I50_sujeto.pkl'))
indices_test = np.array(joblib.load('indices_test_W200_I50_sujeto.pkl'))
sorteo = np.array(joblib.load('sorteo_sujetos.pkl'))

/content/drive/My Drive/repo_tesis/archivos_generados_codigos


Entreno el modelo

In [4]:
start_time = time.time()

# implemento gbm
clf = LGBMClassifier()
clf.fit(features[indices_train, :], label[indices_train, 1], eval_set=[(features[indices_val, :], label[indices_val, 1])], eval_metric='logloss')

end_time = time.time()
training_time = end_time - start_time
# Obtener las iteraciones del mejor modelo
bst = clf.best_iteration_

nombre = 'ronda1_exp_gbm_sep_sub_pca95_n1'

joblib.dump(clf, nombre + '.pkl')

y_pred = clf.predict(features[indices_test, :])

accuracy = accuracy_score(label[indices_test, 1], y_pred)
precision = precision_score(label[indices_test, 1], y_pred, average='weighted')
recall = recall_score(label[indices_test, 1], y_pred, average='weighted')
conf_matrix = confusion_matrix(label[indices_test, 1], y_pred)
plt.savefig("confusion_matrix.png")

API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

exp = Experiment(api_key=API_KEY,
                project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                auto_param_logging=False)
exp.set_name(nombre) # Nombre de este experimento
exp.add_tags(['exp1', 'gbm', 'sep_sub', 'pca95']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(label[indices_test, 1], y_pred)
exp.log_parameter("ind_train", indices_train)
exp.log_parameter("ind_val", indices_val)
exp.log_parameter("ind_test", indices_test)
exp.log_text("Experimento separando los datos por repeticiones y usando GBM. En cuanto a las ventanas: W=200, I=50.")   # Comentario del experimento
exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
exp.end()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058764 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11220
[LightGBM] [Info] Number of data points in the train set: 44634, number of used features: 44
[LightGBM] [Info] Start training from score -2.565061
[LightGBM] [Info] Start training from score -2.601850
[LightGBM] [Info] Start training from score -2.612483
[LightGBM] [Info] Start training from score -2.593124
[LightGBM] [Info] Start training from score -2.582101
[LightGBM] [Info] Start training from score -2.554053
[LightGBM] [Info] Start training from score -2.557806
[LightGBM] [Info] Start training from score -2.557806
[LightGBM] [Info] Start training from score -2.564188
[LightGBM] [Info] Start training from score -2.609129
[LightGBM] [Info] Start training from score -2.299096
[LightGBM] [Info] Start training from score -2.563024
[LightGBM] [Info] Start training from score -2.741400


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/550bfefe3d7c402689a279fa41313ab8

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : ronda1_exp_gbm_sep_sub_pca95_n1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/550bfefe3d7c402689a279fa41313ab8
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.6242177266773395
COMET INFO:     precision     : 0.6375090737728517
COMET INFO:     recall        : 0.6242177266773395
COMET INFO

<Figure size 640x480 with 0 Axes>